In [1]:
cd ../..

e:\GitHub\computervision


In [2]:
import numpy as np
import os
import torch

import torch.nn as nn
import torch.optim as optim

from torchvision import models, transforms
from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING = False
MODEL_WEIGHTS_PATH = 'notebooks/efficientnet/models//'
MODEL_LAST_EPOCH = '.pth'

# CONFIG VARIABLES
DATA_DIR = 'E:/Datasets/cor-splits/sgkf-8-1-1-even'
START_EPOCH = 1 # Change it if you're continuing an existing run
N_EPOCHS = 31
LR = 0.001
LR_EPOCH_SCHEDULE = 10
LR_GAMMA = 0.1
SAVE_INTERVAL = 5 # How many epochs between each backup
PATIENCE = 5
MIN_DELTA = 0

In [4]:
working_folder = 'notebooks/efficientnet'
models_folder = 'models'
from datetime import datetime

curr_time = datetime.now()
curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

output_folder = os.path.join(working_folder, models_folder)
output_folder = os.path.join(output_folder, curr_time)

In [5]:
# efficientnet_b0
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(256, transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(), # When converting to tensor, pytorch automatically rescales to [0,1]
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256, transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [6]:
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)

for param in model.parameters(): # Turning parameters not trainable
    param.requires_grad = True

# A newly defined layer is created with requires_grad=True by default
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=2)
    )

if CONTINUE_TRAINING:
    output_folder = MODEL_WEIGHTS_PATH
    model.load_state_dict(torch.load(output_folder+MODEL_LAST_EPOCH))
else:
    if not os.path.exists(output_folder): 
        os.makedirs(output_folder)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
             
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Only the fully connected layer is being optimized
# optimizer = optim.SGD(model.classifier.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=LR)
optimizer = optim.Adam(model.parameters(), lr=LR)

# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()
step_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)
es = EarlyStopping(patience=PATIENCE, delta=MIN_DELTA)



In [7]:
dataset = PytorchDataset(DATA_DIR, data_transforms)
trainer = PytorchTraining(device, dataset, output_folder)
model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL)

Epoch 1/31
----------
train Loss: 0.7101 Acc: 0.4918
val Loss: 0.6548 Acc: 0.5778
Epoch duration: 0 m 29s
Learning Rate = 0.001
Lower loss found, resetting patience counter

Epoch 2/31
----------
train Loss: 0.6940 Acc: 0.5163
val Loss: 0.6507 Acc: 0.6370
Epoch duration: 0 m 28s
Learning Rate = 0.001
Lower loss found, resetting patience counter

Epoch 3/31
----------
train Loss: 0.6854 Acc: 0.5588
val Loss: 0.6498 Acc: 0.6222
Epoch duration: 0 m 28s
Learning Rate = 0.001
Lower loss found, resetting patience counter

Epoch 4/31
----------
train Loss: 0.6848 Acc: 0.5605
val Loss: 0.6461 Acc: 0.6370
Epoch duration: 0 m 28s
Learning Rate = 0.001
Lower loss found, resetting patience counter

Epoch 5/31
----------
train Loss: 0.6777 Acc: 0.5801
val Loss: 0.6409 Acc: 0.6593
Epoch duration: 0 m 31s
Learning Rate = 0.001
Lower loss found, resetting patience counter

Epoch 6/31
----------
train Loss: 0.6665 Acc: 0.5833
val Loss: 0.6370 Acc: 0.6593
Epoch duration: 0 m 33s
Learning Rate = 0.001
Lo